# Make postISRCCD from raw for Spectractor StandAlone


- work with Weakly_2023_44


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/13
- last update : 2024/07/31


write output file according hierarchy

         DATE/Filter



In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_16/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_16/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_16/Spectractor 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2024_16
   g4213664e8e+c2911b1728 	current w_2024_16 setup


In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits




In [4]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [5]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [6]:
import lsst.daf.butler as dafButler

In [7]:
import warnings
warnings.filterwarnings("ignore")

# Configuration

In [8]:
FLAG_PLOT=True

In [9]:
#DATE = 20230117
#FILTER="empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"

#DATE = 20230118
#FILTER = "BG40_65mm_1-holo4_003"
#FILTER = "OG550_65mm_1-holo4_003"
#FILTER = "empty-holo4_003"

#DATE = 20230119
#FILTER="empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"

#DATE = 20230131
#FILTER = "empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"


#DATE = 20230201
#FILTER = "empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"


#DATE = 20230202
#FILTER = "empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"

#DATE = 20230215
#FILTER="empty-holo4_003"


#DATE = 20230315
#FILTER="empty-holo4_003"

#DATE = 20230316
#FILTER="empty-holo4_003"

#DATE = 20230704
#FILTER = "empty-holo4_003"

#DATE = 20230705
#FILTER = "empty-holo4_003"

#DATE = 20230706
#FILTER = "empty-holo4_003"


#DATE = 20230718
#FILTER = "empty-holo4_003"

#DATE = 20230720
#FILTER = "empty-holo4_003"

#DATE = 20230801
#FILTER = "empty-holo4_003"

#DATE = 20230802
#FILTER = "empty-holo4_003"

#DATE = 20230815
#FILTER = "empty-holo4_003"

#DATE = 20230817
#FILTER = "empty-holo4_003"


#DATE = 20230912
#FILTER="cyl_lens-holo4_001"
#FILTER="cyl_lens-holo4_003"
#FILTER="collimator-holo4_003"
#FILTER="empty-holo4_001"
#FILTER="empty-holo4_003"

#DATE = 20230913
#FILTER="collimator-holo4_003"
#FILTER="cyl_lens-holo4_003"
#FILTER="empty-holo4_003"
#FILTER="empty-holo4_001"

#DATE = 20230914
#FILTER="collimator-holo4_003"
#FILTER="cyl_lens-holo4_003"
#FILTER="empty-holo4_003"
#FILTER="empty-holo4_001"


############### 2021 ##############  In repo main
#-rw-r--r-- 1 dagoret rubin_users 338 Dec 20 01:35 visitdispersers_20211006_filt_BG40-holo4_003.list
#-rw-r--r-- 1 dagoret rubin_users 338 Dec 20 01:35 visitdispersers_20211006_filt_empty-holo4_003.list
#-rw-r--r-- 1 dagoret rubin_users 338 Dec 20 01:35 visitdispersers_20211006_filt_FELH0600-holo4_003.list
#-rw-r--r-- 1 dagoret rubin_users 338 Dec 20 01:35 visitdispersers_20211006_filt_SDSSg-holo4_003.list


#DATE = 20211006
#FILTER="SDSSg-holo4_003"
#FILTER="BG40-holo4_003"
#FILTER="collimator-holo4_003"
#FILTER="empty-holo4_003"

#-rw-r--r-- 1 dagoret rubin_users 442 Dec 20 01:36 visitdispersers_20211103_filt_BG40-holo4_003.list
#-rw-r--r-- 1 dagoret rubin_users 442 Dec 20 01:36 visitdispersers_20211103_filt_empty-holo4_003.list
#-rw-r--r-- 1 dagoret rubin_users 559 Dec 20 01:36 visitdispersers_20211103_filt_FELH0600-holo4_003.list
#-rw-r--r-- 1 dagoret rubin_users 442 Dec 20 01:36 visitdispersers_20211103_filt_SDSSg-holo4_003.list


#DATE = 20211103
#FILTER="SDSSg-holo4_003"
#FILTER="BG40-holo4_003"
#FILTER="collimator-holo4_003"
#FILTER="empty-holo4_003"

#DATE = 20240131
#FILTER="SDSSg-holo4_003"
#FILTER="BG40-holo4_003"
#FILTER="collimator-holo4_003"
#FILTER="empty-holo4_003"


#- 20240709
#- 20240710
#- 20240711
#- 20240715
#- 20240716
#- 20240717
#- 20240724
#- 20240729


DATE = 20240729
#FILTER="collimator-holo4_003"
#FILTER="cyl_lens-holo4_003"
FILTER="empty-holo4_003"
#FILTER="empty-holo4_001"

#filename_visits = f"visitdispersers_{DATE}_filt_{FILTER}.list"
filename_visits = f"all_visitdispersers/{DATE}/visitdispersers_{DATE}_filt_{FILTER}.list"

In [10]:
# output path of type top/date/filter
#----------------------------------------

top_path_out="my_postisrccd_img"
path_out=f"{top_path_out}/{DATE}"

In [11]:
if not os.path.exists(path_out):
    os.makedirs(path_out)

In [12]:
path_out=f"{path_out}/{FILTER}"

if not os.path.exists(path_out):
    os.makedirs(path_out)

# read list of exposures

- generated by ListOfExposures-hologram.ipynb

In [13]:
df = pd.read_csv(filename_visits, header=None,skiprows = 1, sep=' ',names=["date","seq"])
df

,date,seq
0,20240729,278
1,20240729,284
2,20240729,285
3,20240729,291
4,20240729,292
...,...,...
56,20240729,561
57,20240729,567
58,20240729,568
59,20240729,574


In [14]:
for index,row in df.iterrows():
    exposure_selected =row["date"]*100000+row["seq"]
    print(exposure_selected)

2024072900278
2024072900284
2024072900285
2024072900291
2024072900292
2024072900298
2024072900299
2024072900305
2024072900306
2024072900312
2024072900313
2024072900319
2024072900320
2024072900326
2024072900327
2024072900333
2024072900334
2024072900340
2024072900341
2024072900347
2024072900348
2024072900354
2024072900355
2024072900361
2024072900362
2024072900389
2024072900390
2024072900396
2024072900397
2024072900403
2024072900404
2024072900410
2024072900411
2024072900417
2024072900418
2024072900424
2024072900425
2024072900431
2024072900432
2024072900438
2024072900439
2024072900445
2024072900446
2024072900513
2024072900514
2024072900522
2024072900523
2024072900530
2024072900531
2024072900538
2024072900539
2024072900545
2024072900546
2024072900553
2024072900554
2024072900560
2024072900561
2024072900567
2024072900568
2024072900574
2024072900575


## Select flags options

In [15]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Transformations
astropy scale transformations

In [16]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform = PercentileInterval(98.)

## Butler

In [17]:
#repo = '/sdf/group/rubin/repo/main'
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [18]:
collection='LATISS/raw/all'

In [19]:
# configuration
isr_config =  IsrTaskConfig()

In [20]:
isr_config.doDark = False
isr_config.doFlat =  False
isr_config.doFringe = False
isr_config.doDefect = True
isr_config.doLinearize = False
isr_config.doCrosstalk =  False
isr_config.doSaturationInterpolation = False
isr_config.overscan.fitType: 'MEDIAN_PER_ROW'
isr_config.doBias: True


In [21]:
isr_task = IsrTask(config=isr_config)

In [22]:
###########################################
# REPO=/repo/embargo
# butler query-collections $REPO LATISS/calib
#############################################

In [23]:
calibType = 'bias'
physical_filter = 'empty~empty'
cameraName = 'LATISS'
# Collection name containing the verification outputs.
calibCollections = ['LATISS/calib','LATISS/raw/all',]                   

In [24]:
butler = dafButler.Butler(repo, collections=calibCollections)
camera = butler.get('camera', instrument=cameraName)
#bias = butler.get('bias',instrument=cameraName,detector=0)
#defects = butler.get('defects',instrument=cameraName,detector=0)


In [ ]:
%matplotlib inline
if FLAG_PLOT:
    for index,row in df.iterrows():
        exposure_selected =row["date"]*100000+row["seq"]



        raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
        bias = butler.get("bias",instrument=cameraName, exposure= exposure_selected , detector=0, collections=calibCollections)
        defects = butler.get('defects',instrument=cameraName, exposure= exposure_selected ,detector=0,collections=calibCollections)
    
    
            # get metadata
        bias_md = dict(bias.getMetadata())

        try:
            MB_date= bias_md['CALIB_CREATION_DATE']
            MB_time = bias_md['CALIB_CREATION_TIME']
            print(f"Exposure {exposure_selected}, DATE for Master Bias with short calib list",MB_date, MB_time)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            MB_date= bias_md['CALIB_CREATE_DATE']
            MB_time = bias_md['CALIB_CREATE_TIME']
            print(f"Exposure {exposure_selected}, DATE for Master Bias with short calib list",MB_date, MB_time)
    
        defects_md = dict(defects.getMetadata()) 

        try:
            DF_date= defects_md['CALIB_CREATION_DATE']
            DF_time = defects_md['CALIB_CREATION_TIME']
            print(f"Exposure {exposure_selected}, DATE for Master Defect with short calib list",DF_date, DF_time)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            DF_date= defects_md['CALIB_CREATE_DATE']
            DF_time = defects_md['CALIB_CREATE_TIME']
            print(f"Exposure {exposure_selected}, DATE for Master Defect with short calib list",DF_date, DF_time)
    
        meta = raw_img.getMetadata()
        md = meta.toDict()
    
        the_object = md['OBJECT']
        
        if the_object == "MU-COL":
            md['OBJECT']="HD38666"
            the_object = md['OBJECT']
            
        if the_object == "ETA1-DOR":
            md['OBJECT']="HD42525"
            the_object = md['OBJECT']
            
            
        the_am= md['AMSTART']
        the_filter=md['FILTER']
    
        #fast ISR 
        isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
        rotated_array = isr_img.exposure.image.array[::-1,::-1] #rotate the array 180 degrees
        #np.flip(np.flip(a, 1), 0)

    
        fig = plt.figure(figsize=(12,10))
        afw_display = afwDisplay.Display(frame=fig)
        afw_display.scale('linear', 'zscale',None)
        the_title = f"{index} :: isr exposure : {exposure_selected}, target={the_object}, airmass={the_am:.2f}, filter={the_filter}"
        afw_display.mtv(isr_img.exposure.image,title=the_title)

Exposure 2024072900278, DATE for Master Bias with short calib list 2024-03-27 09:39:49 PDT
Exposure 2024072900278, DATE for Master Defect with short calib list 2024-04-16 08:28:20.190150


Exposure 2024072900284, DATE for Master Bias with short calib list 2024-03-27 09:39:49 PDT
Exposure 2024072900284, DATE for Master Defect with short calib list 2024-04-16 08:28:20.190150


In [ ]:
bias_md

In [ ]:
# Save in files

In [ ]:
for index,row in df.iterrows():
    
    exposure_selected =row["date"]*100000+row["seq"]
    
    print(f"exposure selected = {exposure_selected}")


    raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    bias = butler.get("bias",instrument=cameraName, exposure= exposure_selected , detector=0, collections=calibCollections)
    defects = butler.get('defects',instrument=cameraName, exposure= exposure_selected ,detector=0,collections=calibCollections)
    
    
    isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
    arr=isr_img.exposure.image.array
    # 180 degree rotation
    rotated_array = arr[::-1,::-1] #rotate the array 180 degrees
    
    
    meta = raw_img.getMetadata()
    md = meta.toDict()

    the_object = md['OBJECT']
    the_am= md['AMSTART']
    the_filter=md['FILTER']
    
    if the_object == "MU-COL":
        md['OBJECT']="HD38666"
        the_object = md['OBJECT']

    filename_out = f"exposure_{exposure_selected}_pseudo-postisrccd.fits"
    fullfilename_out=os.path.join(path_out,filename_out)
    
    print(f">>>>  output filename {filename_out} object {the_object}")
    
    hdr = fits.Header()
    
    for key,value in md.items():
        if key == 'OBJECT':
            print(key,value)
        hdr[str(key)]=value
        
    # need this    
    hdr["AMEND"] = hdr["AMSTART"]
    
    # be aware weather data may be missing
    if hdr["AIRTEMP"] == None:
        hdr["AIRTEMP"] = 10.0

    if hdr["PRESSURE"] == None:
        hdr["PRESSURE"] = 744.

    if hdr["HUMIDITY"] == None:
        hdr["HUMIDITY"] = 50.

    if hdr["WINDSPD"] == None:
        hdr["WINDSPD"] = 5.

    if hdr["WINDDIR"] == None:
        hdr["WINDDIR"] = 0.   

    if hdr["SEEING"] == None:
        hdr["SEEING"] = 1.15
    
        
   
    
    # Be carefull for Spectractor, 2 hdu units are necessary
    
    primary_hdu = fits.PrimaryHDU(header=hdr)
    image_hdu = fits.ImageHDU(rotated_array)
    
    hdu_list = fits.HDUList([primary_hdu, image_hdu])
    
    hdu_list.writeto(fullfilename_out,overwrite=True)
    
    
    
    #if index>=0:
    #    break
    
